# Importing Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121
import warnings
warnings.simplefilter("ignore")
from PIL import Image
from  tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
import os
Dir = '../input/cassava-leaf-disease-classification'
CFG = {
    'input_dir': "../input/cassava-leaf-disease-classification",
    'train_folder': "train_images",
    'test_folder': "test_images",
}

In [ ]:
train_df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train_df.head()

In [ ]:
train_df['label'].value_counts()

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = [3] * len(train_df.label)
print("The baseline accuracy is {}".format(accuracy_score(y_pred, train_df.label)))

# Model

In [ ]:
Batch_size = 16
img_height, img_width = 300, 300

# Data Augmentation

In [ ]:
train_df['label'] = train_df['label'].astype('str')
gen = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = True,
    validation_split = 0.2,
) 

train_datagen = gen.flow_from_dataframe(
    train_df,
    directory = os.path.join(Dir, "train_images"),
    batch_size = Batch_size,
    target_size = (img_height, img_width),
    subset = "training",
    seed = 42,
    x_col = "image_id",
    y_col = "label",
    class_mode = "categorical"
)

There are 17118 training images.

In [ ]:
val_gen = ImageDataGenerator(
    validation_split = 0.2
)

val_datagen = val_gen.flow_from_dataframe(
    train_df,
    directory = os.path.join(Dir, "train_images"),
    batch_size = Batch_size,
    target_size = (img_height, img_width),
    subset = "validation",
    seed = 42,
    x_col = "image_id",
    y_col = "label",
    class_mode = "categorical"
)

There are 4279 validation images.

In [ ]:
len(train_datagen), len(val_datagen)

**Calculation**:

The length of training images is basically 21397 * 0.8 / 16 = 1070 as the generator returns the batches.

Similarly the length of validation images is 21397 * 0.2 / 16 = 268.

In [ ]:
img, label = next(train_datagen)

next() is used to get the next batch of images and labels.

In [ ]:
Steps_per_train = train_datagen.n / train_datagen.batch_size
Steps_per_val = val_datagen.n / val_datagen.batch_size

# Build model

# Transfer Learning

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB3
def create_model():
    model = models.Sequential()
    model.add(EfficientNetB3(include_top = False, weights = '../input/myefficientnetb3/efficientnetb3_notop.h5',
                             input_shape = (img_height, img_width, 3),  drop_connect_rate=0.3))
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(5, activation='softmax'))
    model.build((300,256))
    loss = tf.keras.losses.CategoricalCrossentropy(
        label_smoothing=0.0001,
        name='categorical_crossentropy'
    )
    model.compile(optimizer = Adam(),
                  loss = loss,
                  metrics = ["categorical_accuracy"])
    return model

model = create_model()
model.layers[0].trainable = False

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

# Training classifier

In [ ]:
rlronp=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                            factor=0.2,
                                            mode = "min",
                                            min_lr=1e-6,
                                            patience=2, 
                                            verbose=1)

estop=tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                       mode= "min",
                                       patience=3, 
                                       verbose=1,
                                       restore_best_weights=True)

history = model.fit_generator(
    train_datagen,
    steps_per_epoch = Steps_per_train,
    epochs = 3,
    validation_data = val_datagen,
    validation_steps = Steps_per_val,
    callbacks = [rlronp,estop]
)


# Fine tuning

In [ ]:
model.layers[0].trainable = True
model.compile(optimizer = Adam(1e-5),
            loss = 'categorical_crossentropy',
            metrics = ["categorical_accuracy"])
history = model.fit_generator(
    train_datagen,
    steps_per_epoch = Steps_per_train,
    epochs = 2,
    validation_data = val_datagen,
    validation_steps = Steps_per_val,
    callbacks = [rlronp,estop]
)
model.save("Casava_Model"+ ".h5")

# Plots between Accuracy and Loss

In [ ]:
import matplotlib.pyplot as plt
train_acc = history.history["categorical_accuracy"]
val_acc = history.history["val_categorical_accuracy"]
epochs = range(1, len(train_acc)+1)
plt.plot(epochs, train_acc, "bo", label = "Training Accuracy")
plt.plot(epochs, val_acc, "b", label = "Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
train_loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(train_loss)+1)
plt.plot(epochs, train_loss, "bo", label = "Training Loss")
plt.plot(epochs, val_loss, "b", label = "Validation Loss")
plt.title("Training and Validation Loss")
plt.legend()

# Submit

In [ ]:
from PIL import Image
import pandas as pd
import numpy as np
from numpy import random
import os
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tqdm
crop_size=300


In [ ]:
def scan_over_image(img_path, crop_size=300):
    '''
    Will extract 512x512 images covering the whole original image
    with some overlap between images
    根据地址读取一张原始图片，剪裁成4张512*512大小的图片
    '''
    
    img = Image.open(img_path)
    img_height, img_width = img.size
    img = np.array(img)
    
    y = random.randint(0,img_height-crop_size)
    x = random.randint(0,img_width-crop_size)

    x_img_origins = [0,img_width-crop_size]
    y_img_origins = [0,img_height-crop_size]
    img_list = []
    for x in x_img_origins:
        for y in y_img_origins:
            img_list.append(img[x:x+crop_size , y:y+crop_size,:])
  
    return np.array(img_list)

In [ ]:
test_time_augmentation_layers = tf.keras.Sequential(
    [
        #反转，缩放，对比度
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomZoom((-0.2, 0)),
        layers.experimental.preprocessing.RandomContrast((0.2,0.2))
    ]
)

In [ ]:
def predict_and_vote(image_filename, folder, TTA_runs=4):
    '''
    Run the model over 4 local areas of the given image,
    before making a decision depending on the most predicted
    disease.
    '''
    
    #apply TTA to each of the 4 images and sum all predictions for each local image
    localised_predictions = []
    local_image_list = scan_over_image(folder+image_filename)
    #将4个剪裁图分别进行4种增强（1张原图对应16张增强图片），
    for local_image in local_image_list:
        #把图片复制成4份 转化成张量类型
        duplicated_local_image = tf.convert_to_tensor(np.array([local_image for i in range(TTA_runs)]))
        #4个图片经过数据增强层得到4个
        augmented_images = test_time_augmentation_layers(duplicated_local_image)
        #将4个图片放入模型预测的到四个结果
        predictions = model.predict(augmented_images)
        #将4个结果的概率相加的到1个剪裁图片的预测
        localised_predictions.append(np.sum(predictions, axis=0))
    
    #sum all predictions from all 4 images and retrieve the index of the highest value
    #将4个剪裁图片相加得到1个
    global_predictions = np.sum(np.array(localised_predictions),axis=0)
    #选出概率最高的那个标签
    final_prediction = np.argmax(global_predictions)
    
    return final_prediction

In [ ]:
def run_predictions_over_image_list(image_list, folder):
    """
    预测数据集
    """
    #用tqdm进度条可视化进度
    predictions = [] 
    with tqdm(total=len(image_list)) as pbar:
        for image_filename in image_list:
            predictions.append(predict_and_vote(image_filename, folder))
            #每完成一张图片预测进度+1
            pbar.update(1)
    return predictions

In [ ]:
#导入测试数据地址
test_folder = '../input/cassava-leaf-disease-classification/test_images/'
#创建提交格式
submission_df = pd.DataFrame(columns={"image_id","label"})
#将测试图片文件名列表写入DF，标签全部设为0
submission_df["image_id"] =  os.listdir(test_folder)
submission_df["label"] = 0

In [ ]:
submission_df["label"] = run_predictions_over_image_list(submission_df["image_id"], test_folder)

In [ ]:
submission_df.to_csv("submission.csv", index=False)